# Preprocessing

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_json("raw.jsonl", lines=True)

## Previewing data

We see that the dataset has 2600 rows of individual movies, with the following columns:
```
'movie_id', 'title', 'release_date', 'budget', 'revenue_worldwide',
'runtime', 'genres', 'imdb_id', 'franchise', 'cast_popularity_mean',
'cast_popularity_max', 'director_popularity', 'original_language',
'imdb_rating', 'imdb_votes', 'mpaa_rating', 'domestic_box_office',
'rotten_tomatoes_score', 'metacritic_score', 'awards_text'
```
Some of these columns contain a significant number of null values.

In [28]:
df.head()

,movie_id,title,release_date,budget,revenue_worldwide,runtime,genres,imdb_id,franchise,cast_popularity_mean,cast_popularity_max,director_popularity,original_language,imdb_rating,imdb_votes,mpaa_rating,domestic_box_office,rotten_tomatoes_score,metacritic_score,awards_text
0,8193,Napoleon Dynamite,2004-06-11,400000,46118097,95,[Comedy],tt0374900,False,1.358067,2.4790,1.3126,en,7.0,251071.0,PG,44540956.0,72.0,64.0,10 wins & 23 nominations total
1,663558,New Gods: Nezha Reborn,2021-02-06,0,70000000,117,"[Animation, Fantasy, Action]",tt13269670,True,0.865500,1.3269,0.5921,zh,6.8,4451.0,TV-14,NaN,80.0,NaN,3 wins & 3 nominations total
2,614409,To All the Boys: Always and Forever,2021-02-12,0,0,115,"[Romance, Comedy, Drama]",tt10676012,True,3.196233,5.6727,0.2037,en,6.3,37385.0,TV-14,NaN,79.0,65.0,2 wins & 2 nominations total
3,12,Finding Nemo,2003-05-30,94000000,940335536,100,"[Animation, Family]",tt0266543,True,1.427567,1.8011,1.2339,en,8.2,1185437.0,G,380843261.0,99.0,90.0,Won 1 Oscar. 49 wins & 63 nominations total
4,16,Dancer in the Dark,2000-09-01,12500000,40061153,140,"[Drama, Crime]",tt0168629,False,1.903567,2.5962,2.2434,en,7.9,122419.0,R,4184036.0,69.0,63.0,Nominated for 1 Oscar. 34 wins & 48 nomination...


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600 entries, 0 to 2599
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movie_id               2600 non-null   int64  
 1   title                  2600 non-null   object 
 2   release_date           2600 non-null   object 
 3   budget                 2600 non-null   int64  
 4   revenue_worldwide      2600 non-null   int64  
 5   runtime                2600 non-null   int64  
 6   genres                 2600 non-null   object 
 7   imdb_id                2600 non-null   object 
 8   franchise              2600 non-null   bool   
 9   cast_popularity_mean   2598 non-null   float64
 10  cast_popularity_max    2598 non-null   float64
 11  director_popularity    2600 non-null   float64
 12  original_language      2600 non-null   object 
 13  imdb_rating            796 non-null    float64
 14  imdb_votes             798 non-null    float64
 15  mpaa

In [30]:
df.shape

(2600, 20)

In [31]:
df.columns

Index(['movie_id', 'title', 'release_date', 'budget', 'revenue_worldwide',
       'runtime', 'genres', 'imdb_id', 'franchise', 'cast_popularity_mean',
       'cast_popularity_max', 'director_popularity', 'original_language',
       'imdb_rating', 'imdb_votes', 'mpaa_rating', 'domestic_box_office',
       'rotten_tomatoes_score', 'metacritic_score', 'awards_text'],
      dtype='object')

## Cleaning

There are a small number of duplicated rows, but most of the work here is imputing missing values.

We decided not to completely drop these columns as the values they have might still be useful.

In [32]:
df.duplicated(subset=["title"]).sum()

np.int64(9)

In [33]:
df.drop_duplicates("title", inplace=True)

In [34]:
(df.revenue_worldwide == 0).sum()

np.int64(705)

In [35]:
df.isnull().sum()

movie_id                    0
title                       0
release_date                0
budget                      0
revenue_worldwide           0
runtime                     0
genres                      0
imdb_id                     0
franchise                   0
cast_popularity_mean        2
cast_popularity_max         2
director_popularity         0
original_language           0
imdb_rating              1796
imdb_votes               1794
mpaa_rating              1791
domestic_box_office      2014
rotten_tomatoes_score    1921
metacritic_score         1961
awards_text              1791
dtype: int64

In [36]:
df['cast_popularity_mean'] = df['cast_popularity_mean'].fillna(df['cast_popularity_mean'].mean())
df['cast_popularity_max'] = df['cast_popularity_max'].fillna(df['cast_popularity_max'].max())

df['imdb_rating'] = df['imdb_rating'].fillna(df['imdb_rating'].median())
df['imdb_votes'] = df['imdb_votes'].fillna(df['imdb_votes'].median())
df['domestic_box_office'] = df['domestic_box_office'].fillna(0)
df['rotten_tomatoes_score'] = df['rotten_tomatoes_score'].fillna(df['rotten_tomatoes_score'].median())
df['metacritic_score'] = df['metacritic_score'].fillna(df['metacritic_score'].median())

df['mpaa_rating'] = df['mpaa_rating'].fillna('Unknown')
df['awards_text'] = df['awards_text'].fillna('No Awards')

In [37]:
# Replace zeros in 'budget' with median
median_val = df[df['budget'] != 0]['budget'].median()
df['budget'] = df['budget'].apply(lambda x: median_val if x == 0 else x)

# Replace zeros in 'revenue_worldwide' with median
median_val = df[df['revenue_worldwide'] != 0]['revenue_worldwide'].median()
df['revenue_worldwide'] = df['revenue_worldwide'].apply(lambda x: median_val if x == 0 else x)

In [38]:
print((df['budget'] == 0).sum())
print((df['revenue_worldwide'] == 0).sum())

0
0


In [39]:
df.isnull().sum()

movie_id                 0
title                    0
release_date             0
budget                   0
revenue_worldwide        0
runtime                  0
genres                   0
imdb_id                  0
franchise                0
cast_popularity_mean     0
cast_popularity_max      0
director_popularity      0
original_language        0
imdb_rating              0
imdb_votes               0
mpaa_rating              0
domestic_box_office      0
rotten_tomatoes_score    0
metacritic_score         0
awards_text              0
dtype: int64

In [40]:
df.dtypes

movie_id                   int64
title                     object
release_date              object
budget                   float64
revenue_worldwide        float64
runtime                    int64
genres                    object
imdb_id                   object
franchise                   bool
cast_popularity_mean     float64
cast_popularity_max      float64
director_popularity      float64
original_language         object
imdb_rating              float64
imdb_votes               float64
mpaa_rating               object
domestic_box_office      float64
rotten_tomatoes_score    float64
metacritic_score         float64
awards_text               object
dtype: object

## Transformation

We convert the `release_date` column into a `season` column as we only need to know which season the movie released in.

In [41]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [42]:
df['release_date'].dtypes

dtype('<M8[ns]')

In [43]:
#Build column for seasons
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['season'] = df['release_date'].dt.month.apply(get_season)

In [44]:
df['season']

0       Summer
1       Winter
2       Winter
3       Spring
4         Fall
         ...  
2595    Winter
2596    Summer
2597      Fall
2598    Spring
2599    Spring
Name: season, Length: 2591, dtype: object

## Dimensionality Reduction

We drop columns which are unneeded.

* `movie_id`, `imdb_id`, `title`: we only need to identify a movie by its index, so these are not needed.
* `awards_text`, `franchise`: these do not relate to our current study.

In [45]:
columns_to_drop = ['movie_id', 'imdb_id', 'title', 'awards_text', 'franchise']
df_reduced = df.drop(columns=columns_to_drop)

In [46]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2591 entries, 0 to 2599
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   release_date           2591 non-null   datetime64[ns]
 1   budget                 2591 non-null   float64       
 2   revenue_worldwide      2591 non-null   float64       
 3   runtime                2591 non-null   int64         
 4   genres                 2591 non-null   object        
 5   cast_popularity_mean   2591 non-null   float64       
 6   cast_popularity_max    2591 non-null   float64       
 7   director_popularity    2591 non-null   float64       
 8   original_language      2591 non-null   object        
 9   imdb_rating            2591 non-null   float64       
 10  imdb_votes             2591 non-null   float64       
 11  mpaa_rating            2591 non-null   object        
 12  domestic_box_office    2591 non-null   float64       
 13  rotten_t

In [47]:
df_reduced.to_csv('movies_cleaned.csv', index=False)